In [ ]:
# Identify column headers
path = 'expanded_shuttle_three_days.csv'

import csv
with open(path, newline='') as f:
    reader = csv.reader(f)
    row1 = next(reader)

columns = row1

In [ ]:
import pandas as pd

dtypes = {'SHUTTLE_COMPANY': str, 'RESTRICTION': str, 'ARTERIAL': str,
          'STREET': str}
df = pd.read_csv(path, error_bad_lines=False, names=columns, header=None, dtype=dtypes)

In [ ]:
# Remove 1st roow
df = df.iloc[1:]

# Change type to datetime
df['TIMESTAMPLOCAL'] = pd.to_datetime(df['TIMESTAMPLOCAL'], format='%d-%b-%y %I.%M.%S.%f %p')

---

In [ ]:
import sfmta
import json
from ipywidgets import Dropdown, HBox, VBox, Button, DatePicker, ColorPicker
from ipyleaflet import Map, Polyline, GeoJSON

def clear_map():
    if 'm' in globals():
        m.close()
    else:
        pass
    
def clear_lat_longs():
    if 'lat_longs' in globals():
        lat_longs = None
    else:
        pass
    
def draw_map(lat_longs):
    center = [37.79481, -122.41186]
    zoom = 12

    m = Map(center=center, zoom=zoom)
    m.layout.height = '650px'
    pl = Polyline(locations=lat_longs)
    pl.color = path_color.value
    pl.fill_color = path_color.value
    m.add_layer(pl)
    return m

plates = df['LICENSE_PLATE_NUM'].unique()
plate_dropdown = Dropdown(options=plates, description='Plate')

def on_button_clicked(b):
    clear_map()
    clear_lat_longs
    plate = plate_dropdown.value
    global lat_longs
    lat_longs = sfmta.lat_long_by_plate(df, plate)
    global m 
    m = draw_map(lat_longs)
    display(m)
        
button = Button(description="Draw/Update Map")
button.on_click(on_button_clicked)


# Add restrictions
def show_restrictions(b):
    polygons = 'vehiclerestrictions_wgs.json'
    
    with open(polygons) as f:
        polygons_json = json.load(f)

    global geojson
    geojson = GeoJSON(data=polygons_json)
    m.add_layer(geojson)

button_restrictions = Button(description="Show Restrictions")
button_restrictions.on_click(show_restrictions)


# Remove restrictions
def remove_restrictions(b):
    if 'geojson' in globals():
        m.remove_layer(geojson)
    else:
        pass
    
def download_data(b):
    if 'lat_longs' in globals():
        df = pd.DataFrame(data=lat_longs)
        df.to_excel('output.xlsx')
    else:
        pass
    
button_remove_restrictions = Button(description="Hide Restrictions")
button_remove_restrictions.on_click(remove_restrictions)

path_color = ColorPicker(description='Color')

export_data = Button(description="Download Data")
export_data.on_click(download_data)

#Temp placeholders
start_date = DatePicker(description='Start')
end_date = DatePicker(description='End')
companies = df['SHUTTLE_COMPANY'].unique()
company = Dropdown(options=companies, value='WeDriveU', description='Company')

VBox([HBox([company, button, button_restrictions, button_remove_restrictions]),
      HBox([plate_dropdown, export_data]),
      HBox([path_color]),
      HBox([start_date]),
      HBox([end_date])
     ])